In [1]:
import polars as pl
from pathlib import Path
from polimi.utils._catboost import reduce_polars_df_memory_size

Tensorflow is not available


In [2]:
dpath = Path('../../dataset')

dtype = 'small'
articles = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/articles.parquet')

behaviors_train = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/train/behaviors.parquet')
history_train = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/train/history.parquet')

behaviors_val = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/validation/behaviors.parquet')
history_val = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/validation/history.parquet')

In [3]:
from polimi.utils._urm import _build_batch_ner_interactions, build_ner_mapping, build_user_id_mapping, build_articles_with_processed_ner
ap = build_articles_with_processed_ner(articles)
ap.head(3)

article_id,ner_clusters
i32,list[str]
3034608,"[""willy strube""]"
3036444,"[""erik andersen"", ""mia teglgaard sprotte""]"
3042022,"[""arias"", ""nielsen""]"


In [4]:
ner_mapping = build_ner_mapping(ap)
ner_mapping.head(2)

ner_index,ner
u32,str
0,"""# metoo"""
1,"""# minhistoriem…"


In [5]:
user_id_mapping = build_user_id_mapping(history_train.vstack(history_val))
user_id_mapping.head(2)

user_index,user_id
u32,u32
0,10068
1,10200


In [6]:
user_id_mapping.shape, ner_mapping.shape

((18827, 2), (43774, 2))

In [7]:
ner_interactions = _build_batch_ner_interactions(history_train, ap, user_id_mapping, ner_mapping, 'article_id_fixed')
ner_interactions

1it [00:01,  1.44s/it]

Memory usage of dataframe is 102.19 MB
Memory usage after optimization is: 102.19 MB
Decreased by 0.0%


user_index,ner_index
u32,u32
0,497
0,883
0,1434
0,1688
0,2387
…,…
18826,43362
18826,43689
18826,43707


In [16]:
ap.filter(pl.col('article_id') == 9746105)

article_id,ner_clusters
i32,list[str]


In [13]:
user_id_mapping.filter(pl.col('user_index') == 18826)['article_id_fixed']

ColumnNotFoundError: article_id_fixed

In [14]:
history_train.filter(pl.col('user_id') == 2590637)['article_id_fixed'].explode()

article_id_fixed
i32
9739296
9739065
9740843
9740749
9737154
…
9770551
9770638
9770604


In [23]:
ap_i= ap.with_columns(
    pl.col('ner_clusters').list.eval(pl.element().replace(ner_mapping['ner'], ner_mapping['ner_index']).cast(pl.Int32))
)
ap_i.head(2)

article_id,ner_clusters
i32,list[i32]
3001353,[]
3003065,[]


In [24]:
df = df.with_columns(pl.col('article_id').replace(ap_i['article_id'], ap_i['ner_clusters']).cast(pl.List(pl.Int32)))\
    .rename({'article_id': 'ner_clusters'})
df = reduce_polars_df_memory_size(df)
df.head(2)

Memory usage of dataframe is 117.37 MB
Memory usage after optimization is: 117.37 MB
Decreased by 0.0%


ner_clusters,date,hour,minute
list[i32],date,i8,i8
"[915, 7645, … 37662]",2023-04-27,10,17
"[8308, 8829, … 35645]",2023-04-27,10,18


In [25]:
most_common_ner_per_hour = df.explode('ner_clusters')\
    .rename({'ner_clusters': 'ner_index'})\
    .group_by('date', 'hour')\
    .agg(pl.col('ner_index').max())\
    .sort('date', 'hour')
most_common_ner_per_hour.head(3)

date,hour,ner_index
date,i8,i32
2023-04-27,7,43746
2023-04-27,8,43602
2023-04-27,9,43746


In [42]:
clicked_ids = behaviors_train.select('article_ids_clicked').explode('article_ids_clicked').drop_nulls().unique()['article_ids_clicked']
home_ids = behaviors_train.select('article_id').drop_nulls().unique()['article_id']
clicked_ids.head(2)

article_ids_clicked
i32
9685524
9745793


In [43]:
len([id for id in home_ids.to_numpy() if id not in clicked_ids.to_numpy()])

3670

In [44]:
len([id for id in clicked_ids.to_numpy() if id not in home_ids.to_numpy()])

1395

In [26]:
most_common_ner_per_day = df.explode('ner_clusters')\
    .rename({'ner_clusters': 'ner_index'})\
    .group_by('date')\
    .agg(pl.col('ner_index').max())\
    .sort('date')
most_common_ner_per_day.head(3)

date,ner_index
date,i32
2023-04-27,43772
2023-04-28,43772
2023-04-29,43767


In [27]:
most_common_ner_per_day.plot('date', 'ner_index', kind='line', title='Most common NER per day')

:Curve   [date]   (ner_index)

In [28]:
most_common_ner_per_hour.plot(x='date', y='ner_index', kind='line', title='Most common NER per hour')

:Curve   [date]   (ner_index)